In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import tensorflow_addons as tfa
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_df = pd.read_csv('../input/nlp-getting-started/train.csv')
test_df = pd.read_csv('../input/nlp-getting-started/test.csv')
train_df.drop(labels=['keyword','location','id'],axis=1,inplace=True)
test_df.drop(labels=['keyword','location','id'],axis=1,inplace=True)

In [3]:
VOCAB_SIZE = 24000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_df['text'])

2022-08-03 01:43:16.214210: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-08-03 01:43:16.351901: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [4]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [5]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=tf.metrics.BinaryAccuracy(threshold=0.4))

In [6]:
history = model.fit(x = train_df['text'], y = train_df['target'], epochs=5,
                    validation_split=0.2, batch_size=64)

Epoch 1/5
96/96 [==============================] - 47s 251ms/step - loss: 0.6888 - binary_accuracy: 0.5793 - val_loss: 0.6898 - val_binary_accuracy: 0.5345
Epoch 2/5
96/96 [==============================] - 17s 183ms/step - loss: 0.6571 - binary_accuracy: 0.5793 - val_loss: 0.6277 - val_binary_accuracy: 0.5345
Epoch 3/5
96/96 [==============================] - 16s 165ms/step - loss: 0.5173 - binary_accuracy: 0.6860 - val_loss: 0.5255 - val_binary_accuracy: 0.7702
Epoch 4/5
96/96 [==============================] - 17s 175ms/step - loss: 0.3444 - binary_accuracy: 0.8762 - val_loss: 0.5178 - val_binary_accuracy: 0.7814
Epoch 5/5
96/96 [==============================] - 17s 181ms/step - loss: 0.2429 - binary_accuracy: 0.9248 - val_loss: 0.5694 - val_binary_accuracy: 0.7787


In [7]:
submission = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')
labels = np.array(tf.sigmoid(model.predict(test_df)))
labels[labels>0.5] = 1
labels[labels<=0.5] = 0
submission['target'] = labels
submission = submission.astype(int)
submission.dtypes
submission.to_csv('submission.csv', index=False)
        
                    